# pandas.to_datetime

参考资料:

- [时间序列与日期用法](https://www.pypandas.cn/docs/user_guide/timeseries.html#%E7%BA%B5%E8%A7%88)
- [pd.to_datetime()时间处理函数](https://www.cnblogs.com/cgmcoding/p/13522038.html)

`pandas.to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=True)`

- arg: int, float, str, datetime, list, tuple, 1-d array, Series, DataFrame/dict-like, 要转换为日期时间的对象
- errors='raise': 
    - 如果为'raise'，则无效的解析将引发异常
    - 如果为'coerce'，则将无效解析设置为NaT
    - 如果为'ignore'，则无效的解析将返回输入
- dayfirst: bool, default False
    - 如果arg是str或类似列表，则指定日期解析顺序。
    - 如果为True，则首先解析日期，例如10/11/12解析为2012-11-10。
    - 警告：dayfirst = True并不严格，但更喜欢使用day first进行解析（这是一个已知的错误，基于dateutil的行为）
- yearfirst: bool, default False
    - 如果arg是str或类似列表，则指定日期解析顺序。
    - 如果True，解析日期以年份为第一，则将10/11/12解析为2010-11-12。
    - 如果dayfirst和yearfirst均为True，则yearfirst在前（与dateutil相同）。
    - 警告：yearfirst = True并不严格，但更喜欢使用year first进行解析（这是一个已知的错误，基于dateutil的行为）。
- utc: bool, default None
    - 如果为True，则返回UTC DatetimeIndex（也转换任何支持tz的datetime.datetime对象）,UTC世界协调时间
- format: str, default None
    - 解析时间的strftime，例如“％d/％m/％Y”，请注意，“％f”将一直解析直至纳秒。
- exact: bool, True by default, 精度
    - 如果为True，则需要精确的格式匹配。
    - 如果为False，则允许格式匹配目标字符串中的任何位置。
- unit: str, default 'ns', 单位
    - D,s,ms,us,ns
    - 例如，在unit ='ms'和origin ='unix'（默认值）的情况下，这将计算到unix纪元开始的毫秒数
- origins: calar, default 'unix',  定义参考日期。自该参考日期以来，数值将被解析为单位数（由unit定义）。
    - 如果是“ unix”（或POSIX）时间；原点设置为1970-01-01。
    - 如果为'julian'，则单位必须为'D'，并且原点设置为Julian Calendar的开头。朱利安天数0被指定为从4713年1月1日中午开始的那一天。
    - 如果时间戳可转换，则将原点设置为由原点标识的时间戳。

## dayfirst/yearfirst

In [1]:
import pandas as pd

print(pd.to_datetime('10/11/12', dayfirst=True)) # 日月年
print(pd.to_datetime('10/11/12', yearfirst=True))  # 年月日
print(pd.to_datetime('10/11/12', dayfirst=True, yearfirst=True)) # 年日月
print(pd.to_datetime('10/11/12')) # 默认月日年

# 可以看出，dayfirst 并没有那么严苛，如果不能把第一个数解析为日，就会以 dayfirst 为 False 进行解析。 
print(pd.to_datetime(['14-01-2012', '01-14-2012'], dayfirst=True))

2012-11-10 00:00:00
2010-11-12 00:00:00
2010-12-11 00:00:00
2012-10-11 00:00:00
DatetimeIndex(['2012-01-14', '2012-01-14'], dtype='datetime64[ns]', freq=None)


## errors

In [2]:
# 如果日期不符合时间戳限制，则传递errors='ignore'将返回原始输入，而不引发任何异常。
print(pd.to_datetime('13000101', errors='ignore'))
# 如果日期不符合时间戳限制，则传递errors='ignore'将返回NaT, not a time
# Pandas 用 NaT 表示日期时间、时间差及时间段的空值，代表了缺失日期或空日期的值，类似于浮点数的 np.nan。
print(pd.to_datetime('13000101', errors='coerce'))

13000101
NaT


## format

要实现精准转换，除了传递 datetime 字符串，还要指定 format 参数，指定此参数还可以加速转换速度。

In [3]:
print(pd.to_datetime('2010-01-12', format='%Y-%m-%d'))
print(pd.to_datetime('2010/01/12', format='%Y/%m/%d'))
print(pd.to_datetime('2010-01-12 05:00', format='%Y-%m-%d %H:%M'))

2010-01-12 00:00:00
2010-01-12 00:00:00
2010-01-12 05:00:00


## origin

origin 参数可以指定 DatetimeIndex 的备选开始时点。例如，把1960-01-01 作为开始日期：

In [4]:
pd.to_datetime([1, 2, 3], unit='D', origin=pd.Timestamp('1960-01-01'))

DatetimeIndex(['1960-01-02', '1960-01-03', '1960-01-04'], dtype='datetime64[ns]', freq=None)

## unit

### 使用Unix纪元时间

In [5]:
pd.to_datetime(1490195805, unit='s')

Timestamp('2017-03-22 15:16:45')

### 使用非Unix纪元时间

In [6]:
pd.to_datetime([1, 2, 3], unit='D', origin=pd.Timestamp('1960-01-01'))

DatetimeIndex(['1960-01-02', '1960-01-03', '1960-01-04'], dtype='datetime64[ns]', freq=None)

## 将年月日合并

In [7]:
import pandas as pd

df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5]})
print(pd.to_datetime(df))

# 只传递组合所需的列也可以
print(pd.to_datetime(df[['year', 'month', 'day']]))

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]
0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]


# pandas.Series.dt

当我们只需要时间数据里面的某些数据时，比如只要年份，可以选择此方法。

- date: 日期
- year: 年
- month: 月
- day: 日
- quarter: 季度
- hour: 小时
- minute: 分钟
- second: 秒

In [8]:
data = pd.Series(pd.date_range("2000-01-01", periods=3, freq="s"))
print(data)
print(data.dt.date)

0   2000-01-01 00:00:00
1   2000-01-01 00:00:01
2   2000-01-01 00:00:02
dtype: datetime64[ns]
0    2000-01-01
1    2000-01-01
2    2000-01-01
dtype: object


In [9]:
data = pd.Series(pd.date_range("2000-01-01", periods=3, freq="q"))
print(data)
print(data.dt.date)
print(data.dt.year)
print(data.dt.month)
print(data.dt.quarter)
print(data.dt.minute)

0   2000-03-31
1   2000-06-30
2   2000-09-30
dtype: datetime64[ns]
0    2000-03-31
1    2000-06-30
2    2000-09-30
dtype: object
0    2000
1    2000
2    2000
dtype: int64
0    3
1    6
2    9
dtype: int64
0    1
1    2
2    3
dtype: int64
0    0
1    0
2    0
dtype: int64


# datetime.datetime

## str转换为datetime

In [10]:
from datetime import datetime

a = datetime.strptime('2015-6-1 18:19:59', '%Y-%m-%d %H:%M:%S')
print(a)

2015-06-01 18:19:59


## datetime转换为str

In [11]:
from datetime import datetime

a = datetime.strptime('2015-6-1 18:19:59', '%Y-%m-%d %H:%M:%S')
# 只保留年月
print(datetime.strftime(a, '%Y-%m'))
# 对pandas处理就结合apply
# df['关闭月份']= df['关闭时间'].apply(lambda x: datetime.datetime.strftime(x, "%Y-%m"))

2015-06


In [13]:
import math
math.ceil(5.3)

6